# Decrypting Signal Backup

In [ ]:
from cryptography.hazmat.primitives          import hashes, hmac
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.backends            import default_backend
from cryptography.hazmat.primitives.ciphers  import Cipher, algorithms, modes

import Backups_pb2
BACKUP_PASSPHRASE = '770374738950407044994423070922'
BACKUP_FILE       = 'signal.backup'

def getBackupKey(passphrase, salt):

    i = bytes(passphrase, encoding='utf-8')
    h = i

    digest = hashes.Hash(hashes.SHA512(), backend=default_backend())
    digest.update(salt)

    for k in range(250000):
        digest.update(h + i)
        h = digest.finalize()
        digest = hashes.Hash(hashes.SHA512(), backend=default_backend())

    return h[:32]

## Load encrypted frames

In [ ]:
data = []
with open(BACKUP_FILE, "rb") as f:
    while True:
        chunk = f.read(4)
        if chunk:
            length = int.from_bytes(chunk, byteorder='big')
            frame = f.read(length)
            mac = frame[-10:]
            frame = frame[:-10]
            data.append([frame, mac])
        else:
            break

#### Get AES init vector and passphrase salt

In [ ]:
headerFrame = data[0][0] + data[0][1]

frame = Backups_pb2.BackupFrame()
frame.ParseFromString(headerFrame)

iv   = frame.header.iv
salt = frame.header.salt

## Decrypt frames

In [ ]:
key = getBackupKey(BACKUP_PASSPHRASE, salt)

derived = HKDF(
    algorithm=hashes.SHA256(),
    length=64,
    salt=None,
    info=bytes('Backup Export', encoding='utf-8'),
    backend=default_backend()
).derive(key)

cipherKey, macKey = derived[0:32], derived[32:64]

In [ ]:
counter = int.from_bytes(iv[:4], byteorder='big')

frames = []

for packet in data[1:]:
    
    # TODO: loop will break when it hits an attachment block

    cipherText = packet[0]
    theirMac   = packet[1]
    
    # Verify MAC
    
    mac = hmac.HMAC(macKey, hashes.SHA256(), backend=default_backend())
    mac.update(cipherText)
    ourMac = mac.finalize()


    if theirMac != ourMac[:10]:
        raise ValueError
        
    # Decrypt

    iv = counter.to_bytes(length=4, byteorder='big') + iv[4:]
    counter += 1
    cipher = Cipher(algorithms.AES(cipherKey),modes.CTR(iv),backend=default_backend()).decryptor()
    plaintext = cipher.update(cipherText) + cipher.finalize()
    
    # Setup protobuf

    frame = Backups_pb2.BackupFrame()
    frame.ParseFromString(plaintext)

    frames.append(frame)